In [1]:
# 整理 json 使用的工具
import csv
import json
# 執行 command 的時候用的
import os
# 匯入 正規表達式
import re
# 子處理程序，用來取代 os.system 的功能
import subprocess
import time
from pathlib import Path
# 強制等待 (執行期間休息一下)
from time import sleep

import numpy as np
import pandas as pd

In [2]:
folderPath = Path(__file__).resolve().parent/'judgement_file'

NameError: name '__file__' is not defined

In [5]:
folderPath = '/Users/allentsai/python_web_scraping-master/judgement_file'

In [3]:
crimeSentencePath = '/Users/allentsai/python_web_scraping-master/judgement_file/judge案由分類/Law_of_the_sentence/Law_of_The_Sentence.csv'

In [4]:
crimeFeaturePath = '/Users/allentsai/python_web_scraping-master/judgement_file/judge案由分類/Law_of_the_sentence/Judgement_Feature2.csv'

In [6]:
crimeSentence = []

In [7]:
csvF = pd.read_csv(crimeFeaturePath,encoding='utf-8')
csvF

,judge_court,judge_year,judge_month,judge_index,judge_title,F_01,F_02,F_03,F_04,F_05,...,judge_crime,judge_crimeClassify,crimeCategory,Match,judge_result,judge_resultInt,judge_class,judge_class2,judge_class3,light_heavy
0,臺灣彰化地方法院,111,1,4,過失傷害,1,1,0,0,0,...,汽車駕駛人無駕駛執照駕車過失傷害罪,過失傷害,1,0,拘役參拾伍日,35,2,1,1,light
1,臺灣彰化地方法院,111,1,6,傷害致重傷害,0,2,0,0,0,...,傷害人致重傷罪,傷害致重傷,4,1,有期徒刑壹年捌月,605,4,1,3,light
2,臺灣彰化地方法院,111,1,18,重傷害,0,0,0,0,0,...,傷害罪,傷害,3,0,有期徒刑陸月,180,3,1,2,light
3,臺灣彰化地方法院,111,1,19,傷害,0,0,0,0,0,...,無罪,無罪,0,1,零日,0,0,0,0,0
4,臺灣彰化地方法院,111,1,27,傷害,0,0,0,0,0,...,無罪,無罪,0,1,零日,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65529,臺灣南投地方法院,109,2,25,過失傷害,0,2,0,0,0,...,過失傷害罪,過失傷害,1,0,拘役肆拾伍日,45,2,1,1,light
65530,臺灣南投地方法院,109,2,27,傷害,0,0,0,0,0,...,傷害罪,傷害,3,0,拘役貳拾日,20,1,1,1,light
65531,臺灣南投地方法院,109,3,2,傷害,0,0,0,0,0,...,傷害罪,傷害,3,0,拘役貳拾伍日,25,1,1,1,light
65532,臺灣南投地方法院,109,3,3,傷害,0,1,0,0,0,...,傷害罪,傷害,3,0,有期徒刑肆月,120,3,1,2,light


In [8]:
csvS = pd.read_csv(crimeSentencePath,encoding='utf-8')
csvS

,crime,crimeCategory,law_of_the_sentence,min,max
0,過失傷害,1,處一年以下有期徒刑、拘役,0,365
1,傷害致重傷,4,處三年以上十年以下有期徒刑,1095,3650
2,傷害,3,處五年以下有期徒刑、拘役,0,1825
3,無罪,0,NaN,0,0
4,過失傷害致重傷,2,處三年以下有期徒刑、拘役,0,1095
5,傷害致死,5,處無期徒刑或七年以上有期徒刑,2555,-1
6,傷害直系血親尊親屬,9,加重其刑至二分之一,0,2737
7,重傷害,6,五年以上十二年以下有期徒刑,1825,4380
8,重傷害未遂,7,NaN,1825,4380
9,其它,11,處六月以上五年以下有期徒刑,180,1825


In [9]:
def class2Percentagy(result,minint,maxint):
    if minint == 0 and maxint == 0:
        percent = 0
    elif maxint == -1:
        percent = (result-minint)/minint
    else:
        percent = (result-minint)/(maxint-minint)
    return f'{(percent*100):.2f}'

In [10]:
for f in range(len(csvF)):
    for s in range(len(csvS)):
        if csvF.loc[f,'crimeCategory'] == csvS.loc[s,'crimeCategory']:
            minint = csvS.loc[s,'min']
            maxint = csvS.loc[s,'max']
            result = csvF.loc[f,'judge_resultInt']
            percent = class2Percentagy(result,minint,maxint)
            crimeSentence.append(percent)

crimeSentence

['9.59',
 '-19.18',
 '9.86',
 '0.00',
 '0.00',
 '4.11',
 '24.66',
 '13.70',
 '24.66',
 '15.89',
 '10.96',
 '13.70',
 '0.00',
 '16.44',
 '5.48',
 '10.96',
 '9.86',
 '-14.29',
 '4.93',
 '15.07',
 '16.44',
 '13.70',
 '4.93',
 '32.88',
 '15.07',
 '16.44',
 '8.22',
 '13.70',
 '16.44',
 '24.66',
 '1.10',
 '3.29',
 '9.59',
 '9.59',
 '16.44',
 '41.10',
 '13.70',
 '10.96',
 '1.10',
 '8.22',
 '2.19',
 '3.29',
 '5.48',
 '24.66',
 '13.70',
 '2.74',
 '8.22',
 '24.66',
 '2.19',
 '32.88',
 '24.66',
 '24.66',
 '0.00',
 '9.86',
 '13.70',
 '8.22',
 '8.22',
 '15.07',
 '24.66',
 '3.01',
 '2.19',
 '10.96',
 '24.66',
 '1.10',
 '10.96',
 '0.00',
 '24.66',
 '2.19',
 '13.70',
 '13.70',
 '4.11',
 '10.96',
 '16.44',
 '13.70',
 '16.44',
 '44.29',
 '10.96',
 '8.22',
 '1.10',
 '24.66',
 '16.44',
 '15.07',
 '24.66',
 '2.74',
 '0.00',
 '16.44',
 '0.00',
 '2.74',
 '41.10',
 '15.07',
 '2.74',
 '8.22',
 '10.96',
 '6.58',
 '15.07',
 '3.01',
 '13.70',
 '10.96',
 '13.70',
 '32.88',
 '3.29',
 '1.10',
 '2.19',
 '13.70',
 '0.

In [12]:
csvF['percentagy%'] = crimeSentence
csvF

,judge_court,judge_year,judge_month,judge_index,judge_title,F_01,F_02,F_03,F_04,F_05,...,judge_crimeClassify,crimeCategory,Match,judge_result,judge_resultInt,judge_class,judge_class2,judge_class3,light_heavy,percentagy%
0,臺灣彰化地方法院,111,1,4,過失傷害,1,1,0,0,0,...,過失傷害,1,0,拘役參拾伍日,35,2,1,1,light,9.59
1,臺灣彰化地方法院,111,1,6,傷害致重傷害,0,2,0,0,0,...,傷害致重傷,4,1,有期徒刑壹年捌月,605,4,1,3,light,-19.18
2,臺灣彰化地方法院,111,1,18,重傷害,0,0,0,0,0,...,傷害,3,0,有期徒刑陸月,180,3,1,2,light,9.86
3,臺灣彰化地方法院,111,1,19,傷害,0,0,0,0,0,...,無罪,0,1,零日,0,0,0,0,0,0.00
4,臺灣彰化地方法院,111,1,27,傷害,0,0,0,0,0,...,無罪,0,1,零日,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65529,臺灣南投地方法院,109,2,25,過失傷害,0,2,0,0,0,...,過失傷害,1,0,拘役肆拾伍日,45,2,1,1,light,12.33
65530,臺灣南投地方法院,109,2,27,傷害,0,0,0,0,0,...,傷害,3,0,拘役貳拾日,20,1,1,1,light,1.10
65531,臺灣南投地方法院,109,3,2,傷害,0,0,0,0,0,...,傷害,3,0,拘役貳拾伍日,25,1,1,1,light,1.37
65532,臺灣南投地方法院,109,3,3,傷害,0,1,0,0,0,...,傷害,3,0,有期徒刑肆月,120,3,1,2,light,6.58


In [17]:
pd.DataFrame(csvF).to_csv(f'{folderPath}/judge案由分類/Judgement_Feature3.csv',encoding='utf-8',index=None)

In [19]:
df = pd.read_csv('/Users/allentsai/python_web_scraping-master/judgement_file/judge案由分類/Judgement_Feature3.csv',encoding='utf-8')
df

,judge_court,judge_year,judge_month,judge_index,judge_title,F_01,F_02,F_03,F_04,F_05,...,judge_crimeClassify,crimeCategory,Match,judge_result,judge_resultInt,judge_class,judge_class2,judge_class3,light_heavy,percentagy%
0,臺灣彰化地方法院,111,1,4,過失傷害,1,1,0,0,0,...,過失傷害,1,0,拘役參拾伍日,35,2,1,1,light,9.59
1,臺灣彰化地方法院,111,1,6,傷害致重傷害,0,2,0,0,0,...,傷害致重傷,4,1,有期徒刑壹年捌月,605,4,1,3,light,-19.18
2,臺灣彰化地方法院,111,1,18,重傷害,0,0,0,0,0,...,傷害,3,0,有期徒刑陸月,180,3,1,2,light,9.86
3,臺灣彰化地方法院,111,1,19,傷害,0,0,0,0,0,...,無罪,0,1,零日,0,0,0,0,0,0.00
4,臺灣彰化地方法院,111,1,27,傷害,0,0,0,0,0,...,無罪,0,1,零日,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65529,臺灣南投地方法院,109,2,25,過失傷害,0,2,0,0,0,...,過失傷害,1,0,拘役肆拾伍日,45,2,1,1,light,12.33
65530,臺灣南投地方法院,109,2,27,傷害,0,0,0,0,0,...,傷害,3,0,拘役貳拾日,20,1,1,1,light,1.10
65531,臺灣南投地方法院,109,3,2,傷害,0,0,0,0,0,...,傷害,3,0,拘役貳拾伍日,25,1,1,1,light,1.37
65532,臺灣南投地方法院,109,3,3,傷害,0,1,0,0,0,...,傷害,3,0,有期徒刑肆月,120,3,1,2,light,6.58
